In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score,f1_score,jaccard_score,precision_score,recall_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

    
def read_image(path):
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    Shape_ori=x.shape
    original_x =x
    x=cv2.resize(x,(W,H))
    x= x/255.0 
    x = x.astype(np.float32)
    x=np.expand_dims(x,axis=0)
    return original_x,x,Shape_ori

def load_data(path):
    # important
    images =sorted(glob(os.path.join(path,"*.png")))
    return images

def save_result(x,y,save_path,shape):
    y_new=y
    y_new=np.expand_dims(y_new,axis=-1)
    

    line = np.ones ((10,shape[1],3)) * 255
    
    ypred = np.concatenate([y_new, y_new, y_new],axis=-1) *255.0
#     print(ypred.shape)
#     print(x.shape)
#     print(line.shape)
#     cat_img = np.concatenate([x,line,ypred],axis=1)
    cat_img = np.concatenate([x,line,ypred],axis=0)
#     print(save_path)
    
    a=cv2.imwrite(save_path,cat_img)
#     print(a)
    
    

In [3]:
smooth = 1e-15
def iou(y_true,y_pred):         # converting simple function to tensor
    def f(y_true,y_pred):
        intersection = (y_true*y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + smooth) / (union + smooth)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f,[y_true,y_pred],tf.float32)

def dice_coef(y_true,y_pred):      # build function using tensor
    y_true = Flatten()(y_true)     # make tensor here itself
    y_pred = Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true*y_pred)
    return (2.*intersection + smooth)/(tf.reduce_sum(y_true)+tf.reduce_sum(y_pred)+smooth)

# Using cross entropy is more of a proxy which is easier to maximize
def dice_loss(y_true,y_pred):       # perform better at class imbalanced problems
    return 1.0 - dice_coef(y_true,y_pred)

# we'll use a specialized loss function that combines binary cross entropy and our dice loss ##################
def bce_dice_loss(y_true,y_pred):
    loss=losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [4]:
with CustomObjectScope({'iou':iou,'dice_coef':dice_coef,'dice_loss':dice_loss,'bce_dice_loss':bce_dice_loss}):
    model = tf.keras.models.load_model("files_new/model.h5")
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 256, 256, 64) 256         conv2d_19[0][0]                  
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 256, 256, 64) 0           batch_normalization_22[0][0]     
_______________________________________________________________________________________________

In [5]:
create_dir("results")

test_dataset_path=os.path.join("testing","image_2")
test_x = load_data(test_dataset_path)


for x in tqdm(test_x):
    name=x.split("\\")[-1].split('.')[0]

    ori_x,x,shape=read_image(x)
    y_pred = model.predict(x)[0] > 0.5 # imp step convert in into 0 and 1
    y_pred = np.squeeze(y_pred , axis=-1)
    y_pred = y_pred.astype(np.float32)
    y_pred = cv2.resize(y_pred,(shape[1],shape[0]))
    save_path = f"results//{name}.png"
    save_result(ori_x,y_pred,save_path,shape)
    


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:01<00:00,  3.25it/s]
